In [3]:
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf

2022-10-23 10:16:37.214572: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [4]:
# import tensorflow dependencies
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, Layer
import tensorflow as tf

In [5]:
#set GPU growth
gpu = tf.config.experimental.list_physical_devices('gpu')
for gpus in gpu:
    tf.config.experimental.set_memory_growth('gpu', True)

2022-10-23 10:16:38.462571: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-10-23 10:16:38.463607: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-10-23 10:16:38.486647: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-10-23 10:16:38.486695: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: a
2022-10-23 10:16:38.486700: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: a
2022-10-23 10:16:38.486870: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 515.65.1
2022-10-23 10:16:38.486887: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 515.65.1
2022-10-23 10:16:38.486891: I tensorflow/stream_executor/cuda/cuda_diagnost

In [6]:
a = np.array(gpu)

In [7]:
# create folder structure
POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negative')
ANC_PATH = os.path.join('data', 'achor')


In [8]:
# make files
os.makedirs(POS_PATH)
os.makedirs(NEG_PATH)
os.makedirs(ANC_PATH)

FileExistsError: [Errno 17] File exists: 'data/positive'

In [ ]:
!tar -xf lfw.tgz

/bin/bash: /home/a/anaconda3/envs/myfirstcode/lib/python3.8/site-packages/cv2/../../../../lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [9]:
for file in os.listdir('lfw'):
    for image in os.listdir(os.path.join('lfw', file)):
        old = os.path.join('lfw', file, image)
        new = os.path.join(NEG_PATH, image)
        os.replace(old, new)

In [10]:
for i, image in enumerate(os.listdir('Images')):
    if i < len(os.listdir('Images'))*.5:
        old = os.path.join('Images', image)
        new = os.path.join(ANC_PATH, image)
        os.replace(old, new)

In [11]:
for i, image in enumerate(os.listdir('Images')):
    old = os.path.join('Images', image)
    new = os.path.join(POS_PATH, image)
    os.replace(old, new)


In [12]:
anchor = tf.data.Dataset.list_files(ANC_PATH+'/*.jpg').take(15)
positive = tf.data.Dataset.list_files(POS_PATH+'/*.jpg').take(15)
negative = tf.data.Dataset.list_files(NEG_PATH+'/*.jpg').take(15)

2022-10-23 10:16:42.632910: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-23 10:16:42.634873: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set


In [13]:
# preprocessing data
def preprocess(file_path):
    #read image
    byte_img = tf.io.read_file(file_path)
    img = tf.io.decode_jpeg(byte_img)

    img1 = tf.image.resize(img, (105, 105))
    img1 = img1/255.0

    return img

In [14]:
converted_positives = []
converted_negatives = []
for i in list(positive):
    i = preprocess(i.numpy())
for j in list(negative):
    j = preprocess(j.numpy())

In [15]:
positives = tf.data.Dataset.zip((anchor, positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives = tf.data.Dataset.zip((anchor, negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))

In [16]:
data = positives.concatenate(negatives)

In [17]:
list(data)[0]

(<tf.Tensor: shape=(), dtype=string, numpy=b'data/achor/WIN_20221021_12_01_26_Pro (2).jpg'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'data/positive/WIN_20221021_11_59_32_Pro (2).jpg'>,
 <tf.Tensor: shape=(), dtype=float32, numpy=1.0>)

In [18]:
# transfer the image_dir to image
def making_image(anchor, valid, label):
    return(preprocess(anchor), preprocess(valid), label)
    

In [19]:
data.map(making_image)

<MapDataset shapes: ((None, None, None), (None, None, None), ()), types: (tf.uint8, tf.uint8, tf.float32)>

In [20]:
data = data.cache()

In [21]:
data = data.shuffle(buffer_size=1000)

In [22]:
# make train data

In [23]:



train_data = data.take(round(len(data)*.7))
train_data = train_data.batch(2)
train_data = train_data.prefetch(8)

In [24]:
test_data = data.take(round(len(data)*.3))
test_data = test_data.batch(2)
test_data = test_data.prefetch(2)

In [25]:
# build model
from unicodedata import name


def embedding():
    inp = Input(shape=(105,105,3), name='input_layer')

    #block1
    c1 = Conv2D(64, (10,10), activation='relu')(inp)
    m1 = MaxPooling2D(64, (2,2), padding='same')(c1)

    #block2
    c2 = Conv2D(128, (7,7), activation='relu')(m1)
    m2 = MaxPooling2D(64, (2,2), padding='same')(c2)

    #block3
    c3 = Conv2D(128, (4,4), activation='relu')(m2)
    m3 = MaxPooling2D(64, (2,2), padding='same')(c3)

    #block 4
    c4 = Conv2D(256, (4,4), activation='relu')(m3)
    f4 = Flatten()(c4)
    d4 = Dense(4096, activation='sigmoid')(f4)

    return Model(inputs = inp, outputs=d4, name='embeddinglayers')

embedding = embedding()


In [26]:
embedding.summary()

Model: "embeddinglayers"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 105, 105, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 96, 96, 64)        19264     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 48, 48, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 42, 42, 128)       401536    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 21, 21, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 18, 18, 128)       262272    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 9, 9, 128)     

In [27]:
class L1Dist(Layer):
    def __init__(self, **kvarg):
        super().__init__()
    
    def call(self, anchor_embedds, validation_embedds):
        return tf.math.abs(anchor_embedds - validation_embedds)



In [28]:
l1 = L1Dist()

In [29]:


def siamesenetwork():
    anchor_input = Input(shape=(105,105,3), name='input_sialayer')
    validate_input = Input(shape=(105,105,3), name='validate_inpsia')

    anchor_embedds = embedding(anchor_input)
    validate_embedds = embedding(validate_input)

    distances = l1(anchor_embedds, validate_embedds)

    d_final = Dense(1, activation='sigmoid')(distances)

    return Model(inputs=[anchor_input, validate_input], outputs=d_final, name ='siamese_network')

    

In [30]:
model = siamesenetwork()

In [31]:
model.summary()

Model: "siamese_network"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_sialayer (InputLayer)     [(None, 105, 105, 3) 0                                            
__________________________________________________________________________________________________
validate_inpsia (InputLayer)    [(None, 105, 105, 3) 0                                            
__________________________________________________________________________________________________
embeddinglayers (Functional)    (None, 4096)         38960448    input_sialayer[0][0]             
                                                                 validate_inpsia[0][0]            
__________________________________________________________________________________________________
l1_dist (L1Dist)                (None, 4096)         0           embeddinglayers[0][

In [32]:
binary_cross_loss = tf.losses.BinaryCrossentropy()

In [33]:
opt = tf.keras.optimizers.Adam(1e-4)

In [35]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(opt=opt, siamesenetwork=model)



In [43]:
@tf.function
def train_step(batch):

    with tf.GradientTape() as tape:
        X = batch[:2]
        y = batch[2]

        yhat = model(X, training=True)

        loss = binary_cross_loss(y, yhat)

        grad = tape.gradient(loss, model.trainable_variables)

        opt.apply_gradients(zip(grad, model.trainable_variables))

    return loss

In [38]:
from tensorflow.keras.metrics import Recall, Precision

In [49]:
def train(data, EPOCHS):
    for epoch in range(1, EPOCHS+1):
        print('\n Epoch {}\{}'.format(epoch, EPOCHS))
        progbar = tf.keras.utils.Progbar(len(data))

        r = Recall()
        p= Precision()

        for idx, batch in enumerate(data):
            loss = train_step(batch)

            yhat = model.predict(batch[:2])

            r.update_state(batch[2], yhat)
            p.update_state(batch[2], yhat)

            progbar.update(idx + 1)
        
        print(loss.numpy(), r.result().numpy(), p.result().numpy())

        if epoch % 10 == 0:
            checkpoint.save(file_prefix = checkpoint_prefix)


In [51]:
EPOCHS = 50


In [ ]:
train(data, EPOCHS)

In [53]:
#evaluate the model
from tensorflow.keras.metrics import Recall, Precision

a = Recall()

test_inp, test_val, y_true = test_data.as_numpy_iterator().next()

In [ ]:
yhat = model.predict(test_inp, test_val)

In [ ]:
[1 if prediction > 0.5 else 0 for prediction in yhat]

In [ ]:
a.update_state(yhat, y_true)

In [59]:
print(a.result().numpy())

0.0


In [60]:
#save model
model.save('siamesemodel.h5')

In [61]:
load_model = tf,keras.models.load_model('siamesemodel.h5', custom_objects={'L1Dist':L1Dist, 'binary_cross_loss':tf.losses.BinaryCrossentropy})